# Tests of New Config System

In [7]:
import importlib
import yaml
from pathlib import Path
import src.config.config as config
importlib.reload(config)

<module 'src.config.config' from '/home/djliden91/git/projects/numerai/src/config/config.py'>

In [8]:
default_config = Path("./src/config/default_config.yaml")
update_config = Path("./src/config/experiments/config_debug.yaml")

In [50]:
class Config:
    def __init__(self):
        self.config = dict()
        self._freeze()
        
    def _freeze(self):
        """Prevent changes to the __dict__"""
        self.__isFrozen = True
        
    def _thaw(self):
        """Allow changes to the __dict__"""
        self.__isFrozen = False

    def update_from_yaml(self, new_config):
        """Update the stored configuration

        kwargs:
        * new_config: path to YAML file

        If new_config includes a key/value pair already included in the
        existing config, the existing config will be overwritten with
        the new value. If new_config includes new key/value pairs, they
        will be added to the existing config.
        """
        self._thaw()
        new_config = yaml.load(open(new_config, 'r'), Loader = yaml.SafeLoader)
        self.config.update(new_config)
        self.__dict__.update(self.config)
        self._freeze()
        
    def update_to_subclass(self, new_config):
        self._thaw()
        new_config = yaml.load(open(new_config, 'r'), Loader = yaml.SafeLoader)
        for key in new_config.keys():
            self.__setattr__(key, Config())
            self.__getattribute__(key).update_from_dict(new_config[key])
        
        
    def update_from_dict(self, new_config:dict):
        """Update the stored configuration

        kwargs:
        * new_config: python dict

        If new_config includes a key/value pair already included in the
        existing config, the existing config will be overwritten with
        the new value. If new_config includes new key/value pairs, they
        will be added to the existing config.
        """
        self._thaw()
        self.config.update(new_config)    
        self.__dict__.update(new_config)
        self._freeze() 

In [142]:
import textwrap

class Config():
    __isFrozen = True
    def __init__(self, config):
        self._thaw()
        self.config = dict()
        if isinstance(config, dict):
            self.update_from_dict(config)
        else:
            self.update_from_yaml(config)
        self._update_state()
        self._freeze()
        
    def _freeze(self):
        """Prevent changes to the __dict__"""
        self.__isFrozen = True
        
    def _thaw(self):
        """Allow changes to the __dict__"""
        self.__isFrozen = False
    
    def _update_state(self):
        for key in self.config.keys():
            if isinstance(self.config[key], dict):
                self.__setattr__(key, Config(self.config[key]))
                
    def __setattr__(self, key, value):
        """Prevent directly setting attributes

        This is to ensure that all configuration options are defined
        directly through the dictionary or YAML adding methods. This
        limitation could be lifted in the future.
        """
        if self.__isFrozen and not hasattr(self, key):
            raise TypeError(textwrap.fill(textwrap.dedent("""
            Directly adding new attributes to a Configuration 
            object using dot notation is not supported. Please use 
            the update_config() method""")))
        object.__setattr__(self, key, value)

                
    def update_config(self, new_config):
        self._thaw()
        if isinstance(new_config, dict):
            self.update_from_dict(new_config)
        else:
            self.update_from_yaml(new_config)
        self._update_state()
        self._freeze()
        
    def update_from_yaml(self, new_config):
        """Update the stored configuration

        kwargs:
        * new_config: path to YAML file

        If new_config includes a key/value pair already included in the
        existing config, the existing config will be overwritten with
        the new value. If new_config includes new key/value pairs, they
        will be added to the existing config.
        """
        new_config = yaml.load(open(new_config, 'r'), Loader = yaml.SafeLoader)
        self.config.update(new_config)
        self.__dict__.update(self.config)
        
    def update_from_dict(self, new_config:dict):
        """Update the stored configuration

        kwargs:
        * new_config: python dict

        If new_config includes a key/value pair already included in the
        existing config, the existing config will be overwritten with
        the new value. If new_config includes new key/value pairs, they
        will be added to the existing config.
        """
        self.config.update(new_config)    
        self.__dict__.update(new_config)

In [143]:
X = Config2(default_config)
X.MODEL.NAME

'Unnamed'

In [144]:
X.update_config(update_config)
X.MODEL.NAME

'config_debug'

In [145]:
cfg_dict = yaml.load(open(update_config, 'r'), Loader=yaml.SafeLoader)
cfg_dict

{'MODEL': {'NAME': 'config_debug', 'WEIGHT_DECAY': 0.5, 'LAYERS': [400, 200]},
 'EVAL': {'SAVE_PREDS': False},
 'TRAIN': {'N_EPOCHS': 2}}

In [146]:
Y = Config2(default_config)
Y.MODEL.NAME

'Unnamed'

In [147]:
Y.update_config(cfg_dict)
Y.MODEL.NAME

'config_debug'

In [149]:
Y.CAT = 'Jasper'

TypeError:  Directly adding new attributes to a Configuration  object using dot
notation is not supported. Please use  the update_config() method

In [155]:
triple_nest = {'A':2,
               'B':{'C':1, 'D':2, 'E':3},
               'C':{'D':{'E':"And we can keep going!"}}}
Y.update_config(triple_nest)
Y.C.D.E

'And we can keep going!'

In [47]:
x.__setattr__('B', {'a':1, 'b':2})

In [49]:
x.__getattribute__('B').items()

dict_items([('a', 1), ('b', 2)])

In [42]:
x.B

123

In [123]:
yaml.load(open(update_config, 'r'), Loader=yaml.SafeLoader)

{'MODEL': {'NAME': 'config_debug', 'WEIGHT_DECAY': 0.5, 'LAYERS': [400, 200]},
 'EVAL': {'SAVE_PREDS': False},
 'TRAIN': {'N_EPOCHS': 2}}

# Try to Figure Out Inheritance

In [26]:
import textwrap
class Froze:
    __isFrozen = True
    def __init__(self):
        self._thaw()
        self.A = 12
        self._freeze()
        
    def _freeze(self):
        """Prevent changes to the __dict__"""
        self.__isFrozen = True
        
    def _thaw(self):
        """Allow changes to the __dict__"""
        self.__isFrozen = False
        
    def __setattr__(self, key, value):
        """Prevent directly setting attributes

        This is to ensure that all configuration options are defined
        directly through the dictionary or YAML adding methods. This
        limitation could be lifted in the future.
        """
        if self.__isFrozen and not hasattr(self, key):
            raise TypeError(textwrap.fill(textwrap.dedent("""
            Directly adding new attributes to a Configuration 
            object using dot notation is not supported. Please use 
            the update_config() method""")))
        object.__setattr__(self, key, value)
        
class Snow(Froze):
    __isFrozen = True
    def __init__(self):
        #self._thaw = super()._thaw
        #self._freeze = super()._freeze
        super().__init__()

In [27]:
s = Snow()

In [30]:
s.B = 13

TypeError:  Directly adding new attributes to a Configuration  object using dot
notation is not supported. Please use  the update_config() method

In [2]:
import os
os.chdir('..')
os.getcwd()
#import ..src.config.config as config
#!cd ..


'/home/djliden91/git/projects/numerai'

In [3]:
from src.config import config

In [4]:
A = config.BaseConfig()

In [9]:
B = config.ConfigI(default_config)

In [10]:
B.A

'This is from the parent class'

In [12]:
B.MODEL.NAME

'Unnamed'

In [13]:
B.update_from_yaml(update_config)

In [14]:
B.MODEL.NAME

AttributeError: 'dict' object has no attribute 'NAME'